In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
from models import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
from fastai.column_data import *

In [5]:
from fastai.conv_learner import *
from fastai.dataset import *

In [6]:
import numpy as np

In [7]:
ls -f KITTI/ | grep .csv

validation.csv
validation_192_640.csv
training.csv
training_192_640.csv


In [8]:
ls KITTI/2011_10_03/2011_10_03_drive_0034_sync/image_03/data/0000003783.png

KITTI/2011_10_03/2011_10_03_drive_0034_sync/image_03/data/0000003783.png


train, val= pd.read_csv('./KITTI/training.csv'), pd.read_csv('./KITTI/validation.csv')

In [9]:
train, val= pd.read_csv('./KITTI/training_192_640.csv'), pd.read_csv('./KITTI/validation_192_640.csv')

In [10]:
for i in range(3):
    train = train.sample(frac=1)
    
for i in range(3):
    val = val.sample(frac=1)

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True);


In [11]:
train.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_09_28/2011_09_28_drive_0043_sync/image_03...,2011_09_28/2011_09_28_drive_0043_sync/image_03...,2011_09_28/2011_09_28_drive_0043_sync/image_03...,364.634611,361.046451,311.532712,92.173583
1,2011_09_26/2011_09_26_drive_0079_sync/image_02...,2011_09_26/2011_09_26_drive_0079_sync/image_02...,2011_09_26/2011_09_26_drive_0079_sync/image_02...,372.106469,368.444783,314.357737,88.265872
2,2011_09_30/2011_09_30_drive_0020_sync/image_02...,2011_09_30/2011_09_30_drive_0020_sync/image_02...,2011_09_30/2011_09_30_drive_0020_sync/image_02...,364.656219,361.067847,310.401186,93.503183
3,2011_09_26/2011_09_26_drive_0039_sync/image_02...,2011_09_26/2011_09_26_drive_0039_sync/image_02...,2011_09_26/2011_09_26_drive_0039_sync/image_02...,372.106469,368.444783,314.357737,88.265872
4,2011_09_30/2011_09_30_drive_0028_sync/image_03...,2011_09_30/2011_09_30_drive_0028_sync/image_03...,2011_09_30/2011_09_30_drive_0028_sync/image_03...,364.656219,361.067847,310.401186,93.503183


In [12]:
val.head()

,t0,t1,t2,fx,fy,cx,cy
0,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,370.723481,367.075404,313.137302,94.57823
1,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,370.723481,367.075404,313.137302,94.57823
2,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,370.723481,367.075404,313.137302,94.57823
3,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,2011_10_03/2011_10_03_drive_0034_sync/image_03...,370.723481,367.075404,313.137302,94.57823
4,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,2011_10_03/2011_10_03_drive_0034_sync/image_02...,370.723481,367.075404,313.137302,94.57823


In [13]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]

In [14]:
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [15]:
trn_camera.head()

,fx,fy,cx,cy
0,364.634611,361.046451,311.532712,92.173583
1,372.106469,368.444783,314.357737,88.265872
2,364.656219,361.067847,310.401186,93.503183
3,372.106469,368.444783,314.357737,88.265872
4,364.656219,361.067847,310.401186,93.503183


val_idxs = get_cv_idxs(len(fnames), val_pct=min(0.01/keep_pct, 0.1))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(fnames), np.array(fnames))
len(val_x),len(trn_x)

In [16]:
aug_tfms = [
    #RandomDihedral(tfm_y=TfmType.NO),
    RandomLighting(b=0.05, c=0.05)]


In [17]:
arch = f
scale,bs = 1, 4
#scale,bs = 4, 32
sz = None #np.array([345, 1242]) 
sz_y = None 

PATH = 'Fastai_TRN'

In [18]:
class UnsupFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        super().__init__(fnames, transform, path)
    def get_y(self, i): return None
    def get_c(self): return 0

In [19]:
class UnShuffleImageData(ImageData):
    def __init__(self, path, datasets, bs, num_workers, classes):
        trn_ds,val_ds,fix_ds,aug_ds,test_ds,test_aug_ds = datasets
        self.path,self.bs,self.num_workers,self.classes = path,bs,num_workers,classes
        self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl,self.test_dl,self.test_aug_dl = [
            self.get_dl(ds,shuf) for ds,shuf in [
                (trn_ds,False),(val_ds,False),(fix_ds,False),(aug_ds,False),
                (test_ds,False),(test_aug_ds,False)
            ]
        ]

In [20]:
class UnScaleTransforms():
    def __init__(self, sz, tfms, normalizer, denorm,
                 tfm_y=TfmType.NO, sz_y=None):
        if sz_y is None: sz_y = sz
        self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        self.tfms = tfms
        if normalizer is not None: self.tfms.append(normalizer)
        self.tfms.append(ChannelOrder(tfm_y))

    def __call__(self, im, y=None): return compose(im, y, self.tfms)
    def __repr__(self): return str(self.tfms)

In [21]:
# just a little modification which remove the multipler at the batch size of the val DL
class UnDBSColumnarModelData(ModelData):
    def __init__(self, path, trn_ds, val_ds, bs, test_ds=None, shuffle=True):
        test_dl = DataLoader(test_ds, bs, shuffle=False, num_workers=1) if test_ds is not None else None
        super().__init__(path, DataLoader(trn_ds, bs, shuffle=shuffle, num_workers=1),
            DataLoader(val_ds, bs, shuffle=False, num_workers=1), test_dl)

    @classmethod
    def from_arrays(cls, path, val_idxs, xs, y, is_reg=True, is_multi=False, bs=64, test_xs=None, shuffle=True):
        ((val_xs, trn_xs), (val_y, trn_y)) = split_by_idx(val_idxs, xs, y)
        test_ds = PassthruDataset(*(test_xs.T), [0] * len(test_xs), is_reg=is_reg, is_multi=is_multi) if test_xs is not None else None
        return cls(path, PassthruDataset(*(trn_xs.T), trn_y, is_reg=is_reg, is_multi=is_multi),
                   PassthruDataset(*(val_xs.T), val_y, is_reg=is_reg, is_multi=is_multi),
                   bs=bs, shuffle=shuffle, test_ds=test_ds)

    @classmethod
    def from_data_frames(cls, path, trn_df, val_df, trn_y, val_y, cat_flds, bs=64, is_reg=True, is_multi=False, test_df=None, shuffle=True):
        trn_ds  = ColumnarDataset.from_data_frame(trn_df,  cat_flds, trn_y, is_reg, is_multi)
        val_ds  = ColumnarDataset.from_data_frame(val_df,  cat_flds, val_y, is_reg, is_multi)
        test_ds = ColumnarDataset.from_data_frame(test_df, cat_flds, None,  is_reg, is_multi) if test_df is not None else None
        return cls(path, trn_ds, val_ds, bs, test_ds=test_ds, shuffle=shuffle)

    @classmethod
    def from_data_frame(cls, path, val_idxs, df, y, cat_flds, bs=64, is_reg=True, is_multi=False, test_df=None, shuffle=True):
        ((val_df, trn_df), (val_y, trn_y)) = split_by_idx(val_idxs, df, y)
        return cls.from_data_frames(path, trn_df, val_df, trn_y, val_y, cat_flds, bs, is_reg, is_multi, test_df=test_df, shuffle=shuffle)

    def get_learner(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                    y_range=None, use_bn=False, **kwargs):
        model = MixedInputModel(emb_szs, n_cont, emb_drop, out_sz, szs, drops, y_range, use_bn, self.is_reg, self.is_multi)
        return StructuredLearner(self, StructuredModel(to_gpu(model)), opt_fn=optim.Adam, **kwargs)


class Transforms():
    def __init__(self, sz, tfms, normalizer, denorm, crop_type=CropType.CENTER,
                 tfm_y=TfmType.NO, sz_y=None):
        if sz_y is None: sz_y = sz
        self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        crop_tfm = crop_fn_lu[crop_type](sz, tfm_y, sz_y)
        self.tfms = tfms
        self.tfms.append(crop_tfm)
        if normalizer is not None: self.tfms.append(normalizer)
        self.tfms.append(ChannelOrder(tfm_y))

    def __call__(self, im, y=None): return compose(im, y, self.tfms)
    def __repr__(self): return str(self.tfms)
    

def image_gen(normalizer, denorm, sz, tfms=None, max_zoom=None, pad=0, crop_type=None,
              tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, scale=None):
    """
    Generate a standard set of transformations

    Arguments
    ---------
     normalizer :
         image normalizing function
     denorm :
         image denormalizing function
     sz :
         size, sz_y = sz if not specified.
     tfms :
         iterable collection of transformation functions
     max_zoom : float,
         maximum zoom
     pad : int,
         padding on top, left, right and bottom
     crop_type :
         crop type
     tfm_y :
         y axis specific transformations
     sz_y :
         y size, height
     pad_mode :
         cv2 padding style: repeat, reflect, etc.

    Returns
    -------
     type : ``Transforms``
         transformer for specified image operations.

    See Also
    --------
     Transforms: the transformer object returned by this function
    """
    if tfm_y is None: tfm_y=TfmType.NO
    if tfms is None: tfms=[]
    elif not isinstance(tfms, collections.Iterable): tfms=[tfms]
    if sz_y is None: sz_y = sz
    if scale is None:
        scale = [RandomScale(sz, max_zoom, tfm_y=tfm_y, sz_y=sz_y) if max_zoom is not None
                 else Scale(sz, tfm_y, sz_y=sz_y)]
    elif not is_listy(scale): scale = [scale]
    if pad: scale.append(AddPadding(pad, mode=pad_mode))
    if crop_type!=CropType.GOOGLENET: tfms=scale+tfms
    return Transforms(sz, tfms, normalizer, denorm, crop_type,
                      tfm_y=tfm_y, sz_y=sz_y)

def noop(x):
    """dummy function for do-nothing.
    equivalent to: lambda x: x"""
    return x

transforms_basic    = [RandomRotate(10), RandomLighting(0.05, 0.05)]
transforms_side_on  = transforms_basic + [RandomFlip()]
transforms_top_down = transforms_basic + [RandomDihedral()]

imagenet_stats = A([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
"""Statistics pertaining to image data from image net. mean and std of the images of each color channel"""
inception_stats = A([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
inception_models = (inception_4, inceptionresnet_2)

def tfms_from_stats(stats, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    """ Given the statistics of the training image sets, returns separate training and validation transform functions
    """
    if aug_tfms is None: aug_tfms=[]
    tfm_norm = Normalize(*stats, tfm_y=tfm_y if norm_y else TfmType.NO) if stats is not None else None
    tfm_denorm = Denormalize(*stats) if stats is not None else None
    val_crop = CropType.CENTER if crop_type in (CropType.RANDOM,CropType.GOOGLENET) else crop_type
    val_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=val_crop,
            tfm_y=tfm_y, sz_y=sz_y, scale=scale)
    trn_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=crop_type,
            tfm_y=tfm_y, sz_y=sz_y, tfms=aug_tfms, max_zoom=max_zoom, pad_mode=pad_mode, scale=scale)
    return trn_tfm, val_tfm


def tfms_from_model(f_model, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    """ Returns separate transformers of images for training and validation.
    Transformers are constructed according to the image statistics given by the model. (See tfms_from_stats)

    Arguments:
        f_model: model, pretrained or not pretrained
    """
    stats = inception_stats if f_model in inception_models else imagenet_stats
    return tfms_from_stats(stats, sz, aug_tfms, max_zoom=max_zoom, pad=pad, crop_type=crop_type,
                           tfm_y=tfm_y, sz_y=sz_y, pad_mode=pad_mode, norm_y=norm_y, scale=scale)


    return Transforms(sz, tfms, normalizer, denorm, crop_type,
                      tfm_y=tfm_y, sz_y=sz_y, )

In [22]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [23]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=sz_y)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=sz_y)

In [24]:
def get_MD(trn, val):
    
#     tfms = tfms_from_model(
#         arch,
#         sz,
#         crop_type=CropType.NO,
#         tfm_y=TfmType.NO,
#         aug_tfms=aug_tfms,
#         sz_y=sz_y)

    tfms = (trn_tfms, val_tfms)
    
    datasets = ImageData.get_ds(
        UnsupFilesDataset,
        (trn, None),
        (val, None),
        tfms, path='KITTI/')
    md = UnShuffleImageData(
        PATH,
        datasets,
        bs, num_workers=16,
        classes=None)
    
    return md

In [25]:
def get_cam(trn_cam, val_cam):
    return UnDBSColumnarModelData.from_data_frames(
        path= None,
        trn_df=trn_cam, val_df=val_cam,
        trn_y=None, val_y=None,
        cat_flds=[],
        bs=bs, shuffle=False)

In [26]:
MD1 = get_MD(trn0, val0)
MD2 = get_MD(trn1, val1)
MD3 = get_MD(trn2, val2)
MDcam = get_cam(trn_camera, val_camera)

In [27]:
class Loss(nn.Module):
    def __init__(self, scale=10, Tscale=2, ndown=2):
        super().__init__()
        self.appr = TriAppearanceLoss(scale=Tscale, ndown=ndown)
        self.smooth = SmoothLoss()
        self.scale = scale
    def forward(self, d1, d2 ,d3, poses_x2, x1, x2, x3, camera):
        appr_loss, details = self.appr(d1, d3, poses_x2, x1, x2, x3, camera)
        smooth_loss = (self.smooth(d1) +self.smooth(d2) + self.smooth(d3)) * self.scale
        #print(type(appr_loss))
        #print(type(smooth_loss))
        return appr_loss + smooth_loss, (appr_loss, smooth_loss, *details) 
               

In [28]:
m = TriDepth(get_base(), 1).cuda()

In [29]:
set_trainable(m.depth.rn, False)

In [30]:
l = Loss(scale=1, Tscale=0.3, ndown=3).cuda() 

In [31]:
#opt = optim.SGD(filter(lambda p: p.requires_grad, m.parameters()), lr=0.001, momentum=0.9)
opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)

imgbatch1 = next(iter(MD1.trn_dl))
imgbatch2 = next(iter(MD2.trn_dl))
imgbatch3 = next(iter(MD3.trn_dl))
cambatch = next(iter(MDcam.trn_dl))

img1 = MD1.trn_ds.denorm(imgbatch1)[0]
img2 = MD2.trn_ds.denorm(imgbatch2)[0]
img3 = MD3.trn_ds.denorm(imgbatch3)[0]
cam = cambatch[1][0]

f, axs = plt.subplots(1, 3, figsize=(12, 6))
axs[0].imshow(img1)
axs[1].imshow(img2)
axs[2].imshow(img3)

print(cam.cpu().numpy().reshape(-1))

DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)

loss, details = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3, cam)
loss.backward()
opt.step()
losses.append(loss)

l1_loss(imgs1, imgs2, mask=V(torch.ones_like(imgs1)))

In [32]:
DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
losses = []
verbose = 20

In [ ]:
for i in range(len(MD1.trn_ds)//bs-len(losses)):
    opt.zero_grad()
    imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
    
    d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)
    loss, details = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3, cam)
    loss.backward()
    opt.step()
    losses.append(loss.data[0])
    # appr smooth ssim l1
    if i%verbose == 0: print(loss.data[0],
                             details[0].data[0],
                             details[1].data[0],
                             details[2].data[0],
                             details[3].data[0],
                             sep='\t')

4.852194309234619	4.663434982299805	0.18875941634178162	2.678974151611328	1.9844608306884766
3.176295518875122	3.0214691162109375	0.1548263281583786	2.245046615600586	0.7764225006103516
3.1700704097747803	3.047956943511963	0.12211351096630096	2.109936237335205	0.9380205869674683
2.671919822692871	2.5678534507751465	0.10406637191772461	1.9295339584350586	0.6383195519447327
2.847351551055908	2.7531020641326904	0.09424948692321777	2.050905704498291	0.7021964192390442
1.9437052011489868	1.8648632764816284	0.078841932117939	1.5586578845977783	0.3062053620815277
2.60164213180542	2.5266942977905273	0.0749477669596672	1.8876267671585083	0.6390675902366638
2.4710490703582764	2.403695821762085	0.06735318154096603	1.8474156856536865	0.5562800765037537
2.337256908416748	2.2758185863494873	0.06143825873732567	1.8279614448547363	0.44785717129707336
2.2940940856933594	2.2389538288116455	0.05514022707939148	1.776921033859253	0.4620327949523926
1.8617500066757202	1.813605785369873	0.048144176602363586	

2.193575859069824	2.181528091430664	0.012047842144966125	1.6824454069137573	0.4990825951099396
2.3219547271728516	2.3085522651672363	0.013402439653873444	1.869402527809143	0.4391498267650604
1.6254668235778809	1.614356279373169	0.011110509745776653	1.294928789138794	0.319427490234375
2.088717460632324	2.074937343597412	0.013780005276203156	1.5284430980682373	0.5464941263198853
2.341287136077881	2.3275222778320312	0.013764899224042892	1.8039705753326416	0.5235515832901001
2.206251621246338	2.1935150623321533	0.012736491858959198	1.721893548965454	0.4716215431690216
2.028201103210449	2.016134262084961	0.01206679455935955	1.6066770553588867	0.4094572961330414
1.9413574934005737	1.9284594058990479	0.012898079119622707	1.5782139301300049	0.3502454161643982
2.1585540771484375	2.1441473960876465	0.014406580477952957	1.639456868171692	0.504690408706665
1.6944553852081299	1.681657314300537	0.01279810257256031	1.3738164901733398	0.30784085392951965
1.954268455505371	1.940231442451477	0.014037035

1.2845377922058105	1.267815113067627	0.016722705215215683	1.0257885456085205	0.24202656745910645
2.0876028537750244	2.068702220916748	0.018900666385889053	1.5708214044570923	0.4978809058666229
2.5271804332733154	2.509502410888672	0.017678022384643555	1.924959659576416	0.5845426917076111
2.2042438983917236	2.1871159076690674	0.017127912491559982	1.5106096267700195	0.6765062212944031
2.014009475708008	1.998408317565918	0.01560111716389656	1.4917430877685547	0.5066651701927185
1.7743018865585327	1.756644368171692	0.017657488584518433	1.4043991565704346	0.35224518179893494
2.208036184310913	2.190098762512207	0.017937464639544487	1.6822160482406616	0.5078827142715454
1.850783109664917	1.835818886756897	0.014964280650019646	1.436429500579834	0.399389386177063
2.067173957824707	2.048800230026245	0.018373781815171242	1.5666682720184326	0.48213186860084534
2.0513362884521484	2.0333609580993652	0.01797521859407425	1.5848331451416016	0.4485277235507965
2.1116321086883545	2.097710371017456	0.01392

1.3859606981277466	1.3700861930847168	0.01587454043328762	1.1480631828308105	0.22202301025390625
1.1084296703338623	1.0928139686584473	0.015615644864737988	0.9065884351730347	0.18622547388076782
1.9402599334716797	1.9235740900039673	0.01668582111597061	1.442719578742981	0.48085451126098633
1.8454340696334839	1.8321071863174438	0.01332683302462101	1.4012515544891357	0.4308556318283081
2.031249761581421	2.013144016265869	0.01810578815639019	1.610154628753662	0.4029894173145294
2.1980879306793213	2.1797304153442383	0.01835745945572853	1.6020076274871826	0.5777226686477661
2.7835192680358887	2.7671406269073486	0.016378657892346382	1.986125111579895	0.7810155153274536
1.847315788269043	1.8318872451782227	0.015428507700562477	1.4476521015167236	0.38423511385917664
1.544432282447815	1.5281543731689453	0.016277913004159927	1.24312424659729	0.2850300967693329
1.6407549381256104	1.6278700828552246	0.012884795665740967	1.2838950157165527	0.3439750373363495
1.6517513990402222	1.638062834739685	0.0

1.702840805053711	1.6872354745864868	0.015605272725224495	1.2905315160751343	0.39670392870903015
2.079563856124878	2.060089588165283	0.01947428286075592	1.6305418014526367	0.42954784631729126
1.7554967403411865	1.736368179321289	0.01912860758602619	1.410749912261963	0.32561829686164856
2.2885661125183105	2.2692229747772217	0.019343184307217598	1.8256245851516724	0.4435984194278717
1.9480589628219604	1.9309117794036865	0.017147228121757507	1.5350134372711182	0.39589837193489075
2.029068946838379	2.0120089054107666	0.017060095444321632	1.5646274089813232	0.44738149642944336
1.7950704097747803	1.7779386043548584	0.017131779342889786	1.4049211740493774	0.37301746010780334
2.1808624267578125	2.162277936935425	0.0185844823718071	1.6831871271133423	0.4790908396244049
1.769093632698059	1.7523348331451416	0.01675884984433651	1.3711285591125488	0.381206214427948
2.3905746936798096	2.372286081314087	0.018288683146238327	1.8560426235198975	0.5162434577941895
1.767077922821045	1.753981351852417	0.0

...

1.4108736515045166	1.3967626094818115	0.014110987074673176	1.0328669548034668	0.36389559507369995
2.052447557449341	2.0375003814697266	0.01494723092764616	1.5736017227172852	0.46389874815940857
2.1533455848693848	2.1386775970458984	0.01466806884855032	1.6664032936096191	0.47227439284324646
1.8319436311721802	1.8172826766967773	0.014660933054983616	1.4421254396438599	0.3751572370529175
1.9428751468658447	1.927621841430664	0.015253296121954918	1.476968765258789	0.4506530165672302
1.397953987121582	1.3848376274108887	0.013116370886564255	1.1051828861236572	0.27965477108955383
2.092137575149536	2.077831268310547	0.014306250959634781	1.6876368522644043	0.3901943266391754
1.9168528318405151	1.9040814638137817	0.012771321460604668	1.4412555694580078	0.4628259241580963
1.7786951065063477	1.764438271522522	0.014256846159696579	1.3438308238983154	0.42060741782188416
2.468790292739868	2.4543137550354004	0.014476533979177475	1.8301920890808105	0.6241217255592346
1.612596035003662	1.597268104553222

1.5156948566436768	1.5009939670562744	0.014700941741466522	1.218742847442627	0.2822510600090027
1.8750355243682861	1.8607834577560425	0.01425209641456604	1.4365065097808838	0.4242769777774811
2.9193315505981445	2.90366268157959	0.01566881313920021	2.038196325302124	0.8654664754867554
1.3868516683578491	1.3735212087631226	0.013330426067113876	1.004591941833496	0.36892926692962646
1.601967453956604	1.5869828462600708	0.014984577894210815	1.2756844758987427	0.31129834055900574
1.6582132577896118	1.6442196369171143	0.013993572443723679	1.3130669593811035	0.3311527371406555
1.5096989870071411	1.4958072900772095	0.013891734182834625	1.1536988019943237	0.34210848808288574
1.9916841983795166	1.97682523727417	0.014858978800475597	1.5366199016571045	0.44020530581474304
1.3528199195861816	1.3386000394821167	0.014219831675291061	1.1187983751296997	0.2198016345500946
2.4658238887786865	2.4508166313171387	0.015007155947387218	1.8505704402923584	0.6002461314201355
1.8014177083969116	1.78764009475708	

1.0423396825790405	1.027372121810913	0.014967596158385277	0.8486013412475586	0.1787707358598709
2.191587448120117	2.1773030757904053	0.014284271746873856	1.7098357677459717	0.4674672484397888
1.9426923990249634	1.9287867546081543	0.013905592262744904	1.5434962511062622	0.3852905333042145
1.5893349647521973	1.575986623764038	0.013348335400223732	1.2389602661132812	0.33702629804611206
1.981533169746399	1.9675565958023071	0.013976624235510826	1.5973758697509766	0.37018075585365295
2.2322239875793457	2.2162857055664062	0.01593836396932602	1.7009350061416626	0.5153508186340332
0.9575934410095215	0.9415440559387207	0.01604936458170414	0.7595174312591553	0.18202662467956543
1.3276054859161377	1.3131731748580933	0.014432299882173538	1.024385929107666	0.28878727555274963
2.358987808227539	2.344851016998291	0.014136707410216331	1.61088228225708	0.7339688539505005
1.8015247583389282	1.787203073501587	0.014321643859148026	1.4895422458648682	0.2976608872413635
1.9313263893127441	1.916632890701294	0

1.9867494106292725	1.973610758781433	0.013138696551322937	1.5813260078430176	0.39228472113609314
1.2510284185409546	1.2367671728134155	0.014261270873248577	0.9548780918121338	0.28188905119895935
2.100457191467285	2.082477569580078	0.017979679629206657	1.6716070175170898	0.41087067127227783
1.954594373703003	1.9400280714035034	0.01456625759601593	1.5360254049301147	0.40400269627571106
2.1982529163360596	2.184382677078247	0.013870345428586006	1.6614363193511963	0.522946298122406
1.5801146030426025	1.566488265991211	0.013626373372972012	1.259640097618103	0.3068481981754303
1.818748116493225	1.8023834228515625	0.01636473275721073	1.4420121908187866	0.3603712320327759
2.048225164413452	2.03361177444458	0.014613272622227669	1.5857231616973877	0.4478885233402252
2.009242296218872	1.9924871921539307	0.016755200922489166	1.5942068099975586	0.3982803523540497
2.0198593139648438	2.0038790702819824	0.015980185940861702	1.5022571086883545	0.5016219019889832
1.2803113460540771	1.2659374475479126	0.0

1.747354507446289	1.7334272861480713	0.013927244581282139	1.3551563024520874	0.3782710134983063
2.269503355026245	2.2554285526275635	0.014074878767132759	1.7485291957855225	0.5068994164466858
1.7023981809616089	1.6886444091796875	0.013753714971244335	1.363830804824829	0.3248136639595032
1.647856593132019	1.6347469091415405	0.013109629973769188	1.2935035228729248	0.3412433862686157
1.6964991092681885	1.681621789932251	0.014877306297421455	1.3522111177444458	0.3294106721878052
1.864579677581787	1.8507916927337646	0.01378798671066761	1.4909462928771973	0.35984545946121216
1.7857049703598022	1.7703148126602173	0.0153901856392622	1.4134721755981445	0.35684260725975037
1.515005350112915	1.5017521381378174	0.013253265991806984	1.272960901260376	0.228791281580925
2.085644245147705	2.071225643157959	0.014418581500649452	1.6285297870635986	0.44269585609436035
1.548482894897461	1.5359137058258057	0.012569157406687737	1.288069486618042	0.24784423410892487
2.107280731201172	2.093327045440674	0.0139

verbose = 20
def fit(model, metric, opt, MD1, MD2, MD3, MDcam):
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []
    for i in range(len(MD1.trn_ds)//bs):
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
        
        d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)
        loss, details = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
        if i%verbose == 0: print(loss.data[0],
                                 details[0].data[0],
                                 details[1].data[0],
                                 details[2].data[0],
                                 details[3].data[0],
                                 sep='\t')
    return A(loesses).mean()

fit(m, l, opt, MD1, MD2, MD3, MDcam)

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)

imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

In [ ]:
d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)

In [ ]:
cx12, cy12, dm12 = l.appr.offset.forward(pose = poses_x2[:,0], inv_depth = d1, camera = cam)
cx32, cy32, dm32 = l.appr.offset.forward(pose = poses_x2[:,1], inv_depth = d3, camera = cam)

x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
x32, ivm32 = l.appr.sampler.forward(imgs2, cx32, cy32) 

In [ ]:
poses_x2.shape

In [ ]:
poses_x2[0,0]

In [ ]:
def plot_depth(depth, index=0, figsize=(12,4), scale=50, inv=True):
    inv_depth = depth.cpu().data[index].numpy()
    inv_depth = np.clip(inv_depth, a_min=0.01, a_max=None)
    if inv:
        depth = 1/inv_depth[0]
    else:
        depth = inv_depth[0]
        
    m, std, mx = depth.mean(axis=(0, 1)), depth.std(axis=(0, 1)), depth.max(axis=(0, 1))
    plt.figure(figsize=figsize)
    plt.imshow(
        depth*scale,
        cmap="viridis",
        #vmin=max(m - 2*std, 0),
        #vmax=min(m+2*std, mx)
    )
    plt.colorbar()
    plt.axis('off')

In [ ]:
def plot_img(recon, index=0, figsize=(12,4)):
    recon = MD1.trn_ds.denorm(recon)[index]
    plt.figure(figsize=figsize)
    plt.imshow(recon)
    plt.axis('off')    

In [ ]:
def plot_mask(mask, index=0, figsize=(12,4)):
    plt.style.use('grayscale')
    mask = mask[index]
    plt.figure(figsize=figsize)
    plt.imshow(mask.cpu().data.numpy(),cmap="gray", vmin=0, vmax=1)
    plt.colorbar()
    plt.axis('off')

In [ ]:
index = 3

In [ ]:
imgs1[0][0]

In [ ]:
plot_img(imgs1, index)

In [ ]:
plot_img(imgs2, index)

In [ ]:
plot_img(x12, index)

In [ ]:
npd1 = 1/(d1.cpu() + 0.01).data.numpy()

In [ ]:
npd1.mean(), npd1.std(), npd1.min(), npd1.max()

In [ ]:
plot_depth(d1, index, scale=1, inv=False)

In [ ]:
plot_mask(ivm12, index)

In [ ]:
plot_mask(dm12, index)

In [ ]:
imgs1[0].mean(), imgs1[0].std()

In [ ]:
d1[0].mean(), d1[0].std()